Visit https://github.com/TrentoCrowdAI/crowdsourced-datasets and follow the instructions to download the datasets.

### TODO

* Handle partially labeled data

In [27]:
import pandas as pd

# dataset_path = "../crowdsourced-datasets/binary-classification/Blue Birds/transformed_dataset.csv"
# dataset_path = "../crowdsourced-datasets/multi-class-classification/Weather Sentiment - AMT/transformed_dataset.csv"
dataset_path = "../crowdsourced-datasets/my-dataset/transformed_dataset.csv"

df = pd.read_csv(dataset_path).dropna(how='all')

In [28]:
# Normalize labels

# Combine unique values
unique_values = pd.Series(pd.unique(df[['response', 'goldLabel']].dropna().values.ravel())).sort_values()

# Map each unique value to a unique integer
value_to_int = pd.Series(range(len(unique_values)), index=unique_values)

# Map column to integers
df['normal_response'] = df['response'].map(value_to_int)
df['normal_goldLabel'] = df['goldLabel'].map(value_to_int)

In [29]:
worker_list = df['workerID'].unique()
task_list = df['taskID'].unique()
label_list = df['normal_goldLabel'].unique()

data = df.groupby('taskID').apply(lambda x: dict(zip(x['workerID'], [x['normal_response']]))).to_dict()
labels_dictionary = df.drop_duplicates(subset='taskID').set_index('taskID')['normal_goldLabel'].to_dict()

/var/folders/6b/b06mlbt913x_d1_xksxw55hr0000gp/T/ipykernel_44390/3466353011.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = df.groupby('taskID').apply(lambda x: dict(zip(x['workerID'], [x['normal_response']]))).to_dict()


In [30]:
print ("num Tasks:", len(task_list))
print ("num Observers:", len(worker_list))
print ("num Classes:", len(label_list))

num Tasks: 7736
num Observers: 3
num Classes: 6


# Run Test

In [31]:
from dawid_skene import run as run_em

_, _, _, _, class_marginals, error_rates, patient_classes = run_em(data, verbose=False)
patient_classes

array([[9.99238190e-01, 7.36524380e-04, 1.74002775e-05, 3.87238289e-57,
        7.88576290e-06],
       [9.78211450e-01, 3.26703162e-03, 1.80738461e-02, 2.50816810e-39,
        4.47671972e-04],
       [9.99238190e-01, 7.36524380e-04, 1.74002775e-05, 3.87238289e-57,
        7.88576290e-06],
       ...,
       [8.50905917e-01, 1.48964424e-01, 1.29658667e-04, 6.49420112e-95,
        2.42370901e-16],
       [5.67728469e-03, 2.04054734e-02, 9.73917242e-01, 1.89676760e-61,
        5.43806694e-15],
       [5.67728469e-03, 2.04054734e-02, 9.73917242e-01, 1.89676760e-61,
        5.43806694e-15]])

# Evaluation

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

In [33]:
true_labels = list(labels_dictionary.values())
estimated_labels = np.argmax(patient_classes, axis=1)

accuracy = int(100 * accuracy_score(true_labels, estimated_labels))

print("Accuracy: %{}".format(accuracy))

Accuracy: %13


In [34]:
# Get the confusion matrices

confusion_matrices = []
for value in df['workerID'].unique():
    worker_df = df[df['workerID'] == value]

    confusion_matrices.append(confusion_matrix(worker_df['normal_goldLabel'], worker_df['normal_response'], labels=label_list))

In [35]:
def frobenius_norm(matrix1, matrix2):
    # Convert the input matrices to numpy arrays if they are not already
    matrix1 = np.array(matrix1)
    matrix2 = np.array(matrix2)
    
    # Calculate the Frobenius norm of the difference
    difference = matrix1 - matrix2
    norm = np.linalg.norm(difference, 'fro')
    
    return norm

In [43]:
print(label_list)
print('confusion matrix 0:\n',confusion_matrices[0],'\n')
print('confusion matrix 1:\n',confusion_matrices[1],'\n')
print('confusion matrix 2:\n',confusion_matrices[2],'\n')


[2 0 1 3 5 4]
confusion matrix 0:
 [[1433    0    0    0    0    0]
 [ 346    0  101  231   77   39]
 [   0    0 4795    0    0    0]
 [   0    0    0  837    0    0]
 [   0    0    0    0  125    0]
 [   0    0    0    0    0   26]] 

confusion matrix 1:
 [[1433    0    0    0    0    0]
 [  71    0  574  144    0    5]
 [   0    0 4795    0    0    0]
 [   0    0    0  837    0    0]
 [   0    0    0    0  125    0]
 [   0    0    0    0    0   26]] 

confusion matrix 2:
 [[1034    0  393    6    0    0]
 [ 121    0  503  121   48    1]
 [ 170    0 4528   92    5    0]
 [   6    0   43  786    2    0]
 [   0    0    2    1  122    0]
 [   0    0    0    0    0   26]] 

error_rates:
 [[[9.50685062e-01 2.71615334e-02 1.93163337e-02 1.08470790e-03
   1.75236298e-03]
  [1.26686189e-01 8.59666101e-01 1.16632556e-02 1.98444433e-03
   9.81399963e-09]
  [4.35631456e-02 1.08910488e-02 9.39156457e-01 5.83350540e-03
   5.55842977e-04]
  [8.54724594e-19 4.80926531e-58 1.14902337e-02 9.88509766e-

In [52]:

print('error_rates:\n',error_rates[0],'\n')

error_rates:
 [[9.50685062e-01 2.71615334e-02 1.93163337e-02 1.08470790e-03
  1.75236298e-03]
 [1.26686189e-01 8.59666101e-01 1.16632556e-02 1.98444433e-03
  9.81399963e-09]
 [4.35631456e-02 1.08910488e-02 9.39156457e-01 5.83350540e-03
  5.55842977e-04]
 [8.54724594e-19 4.80926531e-58 1.14902337e-02 9.88509766e-01
  0.00000000e+00]
 [5.80503620e-02 5.98613888e-14 6.83983199e-02 0.00000000e+00
  8.73551318e-01]] 



In [36]:
parameter_error_rate = np.mean([frobenius_norm(cm_true, cm_estimate) for cm_true, cm_estimate in zip(confusion_matrices, error_rates)])

print("Average Parameter Estimation Error: {:.2f}".format(parameter_error_rate))

ValueError: operands could not be broadcast together with shapes (6,6) (5,5) 